In [25]:
import math
import os
import warnings
from dataclasses import dataclass

import gpytorch
import torch
from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch import Tensor
from torch.quasirandom import SobolEngine

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
tkwargs = {"device": device, "dtype": dtype}

SMOKE_TEST = os.environ.get("SMOKE_TEST")

def get_initial_points(dim, n_pts, seed=0):
    sobol = SobolEngine(dimension=dim, scramble=True, seed=seed)
    X_init = sobol.draw(n=n_pts).to(dtype=dtype, device=device)
    return X_init

In [35]:
import torch
from botorch.test_functions import Ackley
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition import qLogExpectedImprovement
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
import numpy as np
import matplotlib.pyplot as plt

# Define the Ackley function
ackley = Ackley(dim=2,negate=True)

from botorch.models import GenericDeterministicModel
from botorch.models import ModelList
from botorch.generation.sampling import ConstrainedMaxPosteriorSampling

def constraint1(X):
    return X[:, 0] - 1 # Example constraint: x1

def constraint2(X):
    return 2 - X[:, 1] # Example constraint: x2

cmodel1 = GenericDeterministicModel(f=constraint1)
cmodel2 = GenericDeterministicModel(f=constraint2)

constraint_model = ModelList(cmodel1, cmodel2)


# Generate initial training data
train_x = torch.rand(100, 2) * 10 - 5  # 10 random points in [-5, 5]^2
train_y = ackley(train_x).unsqueeze(-1)
# Perform 10 trials
for i in range(10):
    # Fit a GP model
    gp = SingleTaskGP(train_x, train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    # Define the acquisition function
    best_f = train_y.max().item()
    EI = qLogExpectedImprovement(model=gp, best_f=best_f)

    # Optimize the acquisition function
    bounds = torch.tensor([[-5.0, -5.0], [5.0, 5.0]], dtype=torch.float)

    #CMPS = ConstrainedMaxPosteriorSampling(
    #        gp, constraint_model
    #    )
    
    constrained_thompson_sampling = ConstrainedMaxPosteriorSampling(
        model=gp, constraint_model=constraint_model, replacement=False
    )
    print("train_x shape: ", train_x.shape)
    candidate = constrained_thompson_sampling(train_x, num_samples=1)
    #
    #candidate, acq_value = optimize_acqf(
    #    EI, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    #)

    # Evaluate the new candidate
    new_y = ackley(candidate).unsqueeze(-1)

    # Append the new data to the training set
    train_x = torch.cat([train_x, candidate])
    train_y = torch.cat([train_y, new_y])

    print(f"Trial {i+1}: Optimal point: {candidate}")



    #----------------------------------------    plot   ------------------------------------------------------
    

    # Create a grid of points
    x = np.linspace(-5, 5, 100)
    y = np.linspace(-5, 5, 100)
    X, Y = np.meshgrid(x, y)
    Z = np.array([ackley(torch.tensor([x, y])).item() for x, y in zip(X.ravel(), Y.ravel())]).reshape(X.shape)

    # Plot the contour
    plt.contourf(X, Y, Z, levels=50, cmap='viridis')
    plt.colorbar()

    # Scatter plot of training points and the candidate
    plt.scatter(train_x[:, 0].numpy(), train_x[:, 1].numpy(), c='red', label='Training Points')
    plt.scatter(candidate[:, 0].numpy(), candidate[:, 1].numpy(), c='blue', label='Candidate')
    plt.legend()
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.title('Ackley Function Contour with Training Points and Candidate')
    plt.show()

    #----------------------------------------    plot   ------------------------------------------------------

AttributeError: 'GenericDeterministicModel' object has no attribute 'squeeze'

In [22]:
from botorch.models import GenericDeterministicModel
from botorch.models import ModelList
from botorch.generation.sampling import ConstrainedMaxPosteriorSampling

def constraint1(X):
    return X[:, 0] - 1 # Example constraint: x1

def constraint2(X):
    return 2 - X[:, 1] # Example constraint: x2

cmodel1 = GenericDeterministicModel(f=constraint1)
cmodel2 = GenericDeterministicModel(f=constraint2)

constraint_model = ModelList(cmodel1, cmodel2)

CMPS = ConstrainedMaxPosteriorSampling
(
            gp, constraint_model
        )

In [23]:
CMPS

ConstrainedMaxPosteriorSampling(
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
  (objective): IdentityMCObjective()
  (constraint_model): ModelList(
    (models): ModuleList(
      (0-1): 2 x GenericDeterministicModel()
    )
  )
)

In [ ]:
import torch
from botorch.models.deterministic import GenericDeterministicModel
from botorch.generation.sampling import ConstrainedMaxPosteriorSampling
from botorch.acquisition.objective import ConstrainedMCObjective

# 制約関数を定義
def constraint(x):
    return torch.sum(x, dim=-1) - 1

# 制約を満たすかどうかの判定関数
def feasibility(x):
    return constraint(x) <= 0

# 制約付き目的関数を定義
def constrained_objective(samples):
    return samples[..., 0]  # 例として、最初の出力を最大化

# GenericDeterministicModelを使用して制約モデルを作成
f = lambda x: torch.ones_like(x[..., 0])
constraint_model = GenericDeterministicModel(f=f)

constrained_thompson_sampling = ConstrainedMaxPosteriorSampling(
        model=constrained_objective, constraint_model=constraint_model, replacement=False
    )

# サンプリング実行
X = torch.rand(10, 1)  # 入力サンプル
samples = constrained_thompson_sampling(X)

print(samples)

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 2 but got size 1 for tensor number 1 in the list.